In [1]:
import os
import json

from transformers import  RobertaTokenizer

from config import *

In [2]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')

In [3]:
def read_jsonl(path):
    f = open(path, 'r')
    data_ = f.read().split("\n")
    f.close()

    return [json.loads(sample) for sample in data_]
arg = Arg()
data = read_jsonl(arg.test_data_file)

In [4]:
def convert_lines_to_tokens(line, tokenizer):

    code_tokens=tokenizer.tokenize(code)
    source_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
    source_ids =  tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = args.block_size - len(source_ids)
    source_ids+=[tokenizer.pad_token_id]*padding_length
    return InputFeatures(source_tokens, source_ids, js[args.idx_key], js['target'])

In [5]:
def read_explanation_output(path):
    f = open(path, 'r')
    data_ = json.loads(f.read())
    f.close()
    return data_

In [6]:
explanations = read_explanation_output('output.json')

In [7]:
cnt = 0
line_scores = []
for explan, sample in zip(explanations, data):
    assert(int(explan['idx']) == int(sample['id']))
    func = sample['func']
    tokens = explan['tokens'][0]
    scores = explan['scores'][0]
    index = tokens.index("</s>")
    tokens = tokens[1:index]
    scores = scores[1:index]

    no = 0
    line_score = []
    for line in func.split("\n"):
        line_ = ' '.join(line.split())
        if line_ == '' or line_ == '\n' or tokens == []:
            line_score.append(0)
            continue
        if no:
            line_ = ' ' + line_
        no += 1

        line_token = tokenizer.tokenize(line_)
        
        end_ind = min(len(line_token), len(tokens))
        assert tokens[: end_ind] == line_token[:end_ind], tokens[:end_ind]

        score_ = scores[:end_ind]
        line_score.append(sum(score_) / len(score_))

        tokens = tokens[end_ind:]
        scores = scores[end_ind:]
    
    line_scores.append(line_score)


In [8]:
top_10, false_alarm = 0, 0
total_vul = 0
for sample, line_score in zip(data, line_scores):
    if sample['target'] == 0:
        continue
    total_vul += 1
    func = sample['func'].split("\n")
    diff = sample['diff']
    assert len(func) == len(line_score)
    indices = []
    for key in diff:
        ind = int(key)
        assert func[ind] == diff[key]
        indices.append(ind)
    
    ind_score = [(score, ind) for ind, score in enumerate(line_score)]
    ind_score = sorted(ind_score, reverse=True)

    # print(indices)
    for idx, (score, ind) in enumerate(ind_score):
        # print(ind)
        if ind in indices:
            
            cnt = idx
            for k in range(idx):
                if func[k] == '' or func[k] == ' ' or func[k] == '\n':
                    cnt -= 1
            
            false_alarm += cnt / 3
            if cnt < 12:
                top_10 += 1

         
                # print(sample['id'])
            break
            
print('Top 10 ', top_10 / total_vul)
print('False Alarm ', false_alarm / total_vul)

Top 10  0.6566473988439306
False Alarm  8.256647398843926
